<a href="https://colab.research.google.com/github/rebeccahe1998/ScrapeingStuff/blob/main/Count_Call_Analyst.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from bs4 import BeautifulSoup
import requests
import time
import random
import os
import re
import numpy as np
import pandas as pd
#!pip install html2text
import html2text
#!pip install yfinance
import yfinance as yf

1. Get ticker and its full name required for Motley Fool search url
2. Get all the available transcripts urls for the ticker
3. Count how many analysts of text after the key word by splitting keyword 'Call participants'
4. Generate a Excel file

In [2]:
def full_name(ticker):
    return yf.Ticker(ticker).info['longName'].split()[0].split(',')[0]
full_name('UPWP')

'Upwork'

In [3]:
ticker_list = ['TWOU','UPWK']
user_agent = ["Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.121 Safari/537.36",
                "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/7.0.540.0 Safari/534.10",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.4 (KHTML, like Gecko) Chrome/6.0.481.0 Safari/534.4",
                "Mozilla/5.0 (Macintosh; U; Intel Mac OS X; en-US) AppleWebKit/533.4 (KHTML, like Gecko) Chrome/5.0.375.86 Safari/533.4",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/532.2 (KHTML, like Gecko) Chrome/4.0.223.3 Safari/532.2",
                "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/4.0.201.1 Safari/532.0",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/532.0 (KHTML, like Gecko) Chrome/3.0.195.27 Safari/532.0",
                "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US) AppleWebKit/530.5 (KHTML, like Gecko) Chrome/2.0.173.1 Safari/530.5",
                "Mozilla/5.0 (Windows; U; Windows NT 5.2; en-US) AppleWebKit/534.10 (KHTML, like Gecko) Chrome/8.0.558.0 Safari/534.10",
                "Mozilla/5.0 (X11; U; Linux x86_64; en-US) AppleWebKit/540.0 (KHTML,like Gecko) Chrome/9.1.0.0 Safari/540.0"]

In [5]:
def headers():
    return {'User-Agent':user_agent[random.randint(0, len(user_agent) - 1)],}
def get_urls(full_name, ticker):
    url_ = []
    res = requests.get(f'https://www.fool.com/quote/nasdaq/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
    # test which exchange(NYSE OR NASDAQ)
    if not res.status_code == 200:
        try:
            res = requests.get(f'https://www.fool.com/quote/nyse/{full_name}/{ticker}/',
                   headers=headers(),proxies={'http':'150.129.148.88'})
        except: print('URL might be wrong')
    soup = BeautifulSoup(res.content,'lxml')
    for i in soup.find_all('a', {'data-id' : 'article-list-hl'}, href=re.compile("transcripts")):
        url_.append('https://www.fool.com'+i.get('href'))
    return url_


In [12]:
get_urls('2U','TWOU')

['https://www.fool.com/earnings/call-transcripts/2021/02/11/2u-inc-twou-q4-2020-earnings-call-transcript/',
 'https://www.fool.com/earnings/call-transcripts/2020/10/27/2u-inc-twou-q3-2020-earnings-call-transcript/',
 'https://www.fool.com/earnings/call-transcripts/2020/07/31/2u-inc-twou-q2-2020-earnings-call-transcript.aspx',
 'https://www.fool.com/earnings/call-transcripts/2021/02/11/2u-inc-twou-q4-2020-earnings-call-transcript/',
 'https://www.fool.com/earnings/call-transcripts/2020/10/27/2u-inc-twou-q3-2020-earnings-call-transcript/',
 'https://www.fool.com/earnings/call-transcripts/2020/07/31/2u-inc-twou-q2-2020-earnings-call-transcript.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/05/01/2u-inc-twou-q1-2020-earnings-call-transcript.aspx',
 'https://www.fool.com/earnings/call-transcripts/2020/02/07/2u-inc-twou-q4-2019-earnings-call-transcript.aspx',
 'https://www.fool.com/earnings/call-transcripts/2019/11/13/2u-inc-twou-q3-2019-earnings-call-transcript.aspx',
 'https:

In [48]:
def get_call_participated_analyst(ticker):
    full = full_name(ticker) 
    url_ = get_urls(full,ticker)
    inner_url_analyst_count = {'Date':[], 'Count': []}
    for i,inner_url in enumerate(url_):
        time.sleep(np.random.randint(10))
        date = re.findall(r'(\d+/\d+/\d+)',inner_url)[0]
        inner_url_analyst_count['Date'].append(str(date))
        
        res = requests.get(inner_url,headers=headers(),proxies={'http':'150.129.148.88'})
        #except: print('Requests went wrong')
        soup = BeautifulSoup(res.content,'lxml') 
        text = str(res.text)
        text = html2text.html2text(text)
        for i in ['Call Participants','Call participant']:
            text = text.replace(i, 'Call participants')
        participants = text.split('Call participants')[-1] #split by keyword to get participants
        inner_url_analyst_count['Count'].append(participants.count('Analyst'))
    return inner_url_analyst_count

In [44]:
get_call_participated_analyst('TWOU')

{'Count': [29, 20, 19, 29, 20, 19, 20, 19, 21, 35, 35, 41, 51],
 'Date': ['2021/02/11',
  '2020/10/27',
  '2020/07/31',
  '2021/02/11',
  '2020/10/27',
  '2020/07/31',
  '2020/05/01',
  '2020/02/07',
  '2019/11/13',
  '2019/07/30',
  '2019/05/08',
  '2019/02/26',
  '2018/11/05']}

In [49]:
writer = pd.ExcelWriter('Earnings_Call_Analyst_Count.xlsx')
def get_df(ticker):
    df = pd.DataFrame(get_call_participated_analyst(ticker)).reindex(columns = ['Date','Count'])
    df.to_excel(writer, sheet_name=ticker)

get_df('TWOU')
writer.save()